# 🤺 Demo: Using the Environment in Agents and Tools

### 🤔 What is the environment?

When creating an agent, it is possible to pass an environment dictionary to the agent. This environment dictionary can be used to store information that is relevant to the agent, but not part of the model or tools. This information can be accessed by the agent and its delegates and tools during execution. This allows for the agent to be called with a specific set of hardcoded values that are scoped to that agent, e.g. a customer ID, a user ID, or a specific configuration. 

### ⚙️ How does it work?

The environment is passed to the agent when it is created. The agent can then access the environment using the `self.environment` attribute. The environment is a dictionary, so it can store any type of data that is relevant to the agent. The agent, by default, passes this environment to its tools when they are executed. This allows the tools to access the environment as well.

### 📄 Example: Using an orchestrator agent to pass a customer ID to a tool and a delegate agent

In [1]:
# Import the necessary classes
from fence.agents.agent import Agent
from fence.tools.base import BaseTool
from fence.utils.logger import setup_logging
from fence.models.openai import GPT4omini
logger = setup_logging(__name__, log_level='debug', are_you_serious=False)

Let's build a basic tool that retrieves the account holder name from a database. The tool will use the `current_account_id` from the environment to determine which account holder name to return. The tool will return "Bert" if the `current_account_id` is "foo", "Ernie" if the `current_account_id` is "bar", and "Unknown" otherwise.

Remember, tools take their description from either the `description` attribute on init, or the docstring if no description is passed.

In [2]:
class AccountNameRetrieverTool(BaseTool):
    """
    Tool to retrieve the account holder name from a database.
    """
    def execute_tool(self, environment):
        account_id = self.environment.get("current_account_id", "unknown")
        logger.info(f"Retrieving account holder name for account_id: {account_id}")
        if account_id == "foo":
            return "Bert"
        if account_id == "bar":
            return "Ernie"
        return "Unknown"

Now let's build an agent that uses the `AccountNameRetrieverTool` to retrieve the account holder name. The agent will not explicitly need to pass anything to the tool, as the tool will access the `current_account_id` from the environment. The agent will set the `current_account_id` to "foo" and then run the tool to retrieve the account holder name.

In [3]:
# Create an agent with a model and tools
agent = Agent(
    identifier='accountant',
    model=GPT4omini(source="agent"),
    tools=[AccountNameRetrieverTool()],
    environment={"current_account_id": "foo"},
)

agent.run('what is the current account holders name?')    # "How much is 9 + 10?",
    

[2025-03-28 22:05:40] [INFO] ℹ️ [agent.__init__:96] Initialized agent with model <GPT 4o mini>, delegates: [], tools: ['AccountNameRetrieverTool']
[2025-03-28 22:05:40] [DEBUG] 🔵 [gpt._invoke:141] Request body: {'temperature': 1, 'max_tokens': None, 'messages': [{'role': 'user', 'content': 'what is the current account holders name?'}, {'content': 'You are a general purpose agent, capable of delegating tasks to other agents or tools. You will be given input, which starts with a question, and then potentially a series of [OBSERVATION]s, [DELEGATE]s and [ACTION]s. You will need to process this input. Always begin with a [THOUGHT] to help reason what the next best step is.\n Then, continue with one of these options: [ACTION], [DELEGATE] or [ANSWER].\n\nChoose [ACTION] when you need more information, for which you have a tool available. You can call this tool by replying as follows:\n\n[THOUGHT] I need more information to answer the question. I have a tool that can provide me with a piece o

"The current account holder's name is Bert."

Now let's create two agents, one delegating to the other, to verify whether the environment is passed to the delegate. The parent agent will set the `current_account_id` to "bar" and delegate to the child agent. The child agent will use the `AccountNameRetrieverTool` to retrieve the account holder name.

In [4]:
# Create the agents
child_agent = Agent(
    identifier='child_accountant',
    description="Specialized accountant agent for account specifics.",
    model=GPT4omini(source="agent"),
    tools=[AccountNameRetrieverTool()],
)
parent_agent = Agent(
    identifier='parent_accountant',
    model=GPT4omini(source="agent"),
    delegates=[child_agent],
    environment={"current_account_id": "bar"},
)
parent_agent.run('what is the current account holders name?')

[2025-03-28 22:05:42] [INFO] ℹ️ [agent.__init__:96] Initialized agent with model <GPT 4o mini>, delegates: [], tools: ['AccountNameRetrieverTool']
[2025-03-28 22:05:42] [INFO] ℹ️ [agent.__init__:96] Initialized agent with model <GPT 4o mini>, delegates: ['child_accountant'], tools: []
[2025-03-28 22:05:42] [DEBUG] 🔵 [gpt._invoke:141] Request body: {'temperature': 1, 'max_tokens': None, 'messages': [{'role': 'user', 'content': 'what is the current account holders name?'}, {'content': 'You are a general purpose agent, capable of delegating tasks to other agents or tools. You will be given input, which starts with a question, and then potentially a series of [OBSERVATION]s, [DELEGATE]s and [ACTION]s. You will need to process this input. Always begin with a [THOUGHT] to help reason what the next best step is.\n Then, continue with one of these options: [ACTION], [DELEGATE] or [ANSWER].\n\nChoose [ACTION] when you need more information, for which you have a tool available. You can call this

"The current account holder's name is Ernie."